# **Using LibAUC optimizers with Pytorch Learning Rate Scheduling**

**Author**: Zhuoning Yuan

**Introduction**

In this tutorial, you will learn how to quickly train models using LibAUC with [Pytorch Learning Rate Scheduler](https:/https://www.kaggle.com/code/isbhargav/guide-to-pytorch-learning-rate-scheduling/notebook/). After completion of this tutorial, you should be able to use LibAUC to train your own models on your own datasets.

**Useful Resources**:
* Website: https://libauc.org
* Github: https://github.com/Optimization-AI/LibAUC

**Reference**:  

If you find this tutorial helpful in your work,  please acknowledge our library and cite the following paper:

<pre>
@inproceedings{yuan2021large,
  title={Large-scale robust deep auc maximization: A new surrogate loss and empirical studies on medical image classification},
  author={Yuan, Zhuoning and Yan, Yan and Sonka, Milan and Yang, Tianbao},
  booktitle={Proceedings of the IEEE/CVF International Conference on Computer Vision},
  pages={3040--3049},
  year={2021}
  }
</pre>


# **Installing LibAUC**

In [ ]:
!pip install libauc==1.2.0

# **Importing AUC Training Pipeline**

In [6]:
from libauc.losses import AUCMLoss
from libauc.optimizers import PESG
from libauc.models import resnet20 as ResNet20
from libauc.datasets import CIFAR10
from libauc.utils import ImbalancedDataGenerator
from libauc.metrics import auc_roc_score

import torch 
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
from torch.utils.data import Dataset

class ImageDataset(Dataset):
    def __init__(self, images, targets, image_size=32, crop_size=30, mode='train'):
       self.images = images.astype(np.uint8)
       self.targets = targets
       self.mode = mode
       self.transform_train = transforms.Compose([                                                
                              transforms.ToTensor(),
                              transforms.RandomCrop((crop_size, crop_size), padding=None),
                              transforms.RandomHorizontalFlip(),
                              transforms.Resize((image_size, image_size)),
                              ])
       self.transform_test = transforms.Compose([
                             transforms.ToTensor(),
                             transforms.Resize((image_size, image_size)),
                              ])
    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        target = self.targets[idx]
        image = Image.fromarray(image.astype('uint8'))
        if self.mode == 'train':
            image = self.transform_train(image)
        else:
            image = self.transform_test(image)
        return image, target

# paramaters
SEED = 123
BATCH_SIZE = 128
imratio = 0.1
lr = 0.1
epoch_decay = 2e-3 # 1/gamma
weight_decay = 1e-4
margin = 1.0


# dataloader 
(train_data, train_label) = CIFAR10(root='./data', train=True) 
(test_data, test_label) = CIFAR10(root='./data', train=False) 

generator = ImbalancedDataGenerator(verbose=True, random_seed=0)
(train_images, train_labels) = generator.transform(train_data, train_label, imratio=imratio)
(test_images, test_labels) = generator.transform(test_data, test_label, imratio=0.5)

trainloader = torch.utils.data.DataLoader(ImageDataset(train_images, train_labels), batch_size=BATCH_SIZE, shuffle=True, num_workers=1, pin_memory=True, drop_last=True)
testloader = torch.utils.data.DataLoader(ImageDataset(test_images, test_labels, mode='test'), batch_size=BATCH_SIZE, shuffle=False, num_workers=1,  pin_memory=True)

# model 
model = ResNet20(pretrained=False, num_classes=1)
model = model.cuda()

# loss & optimizer
loss_fn = AUCMLoss()
optimizer = PESG(model, 
                 loss_fn=loss_fn,
                 lr=lr, 
                 margin=margin,
                 epoch_decay=epoch_decay, 
                 weight_decay=weight_decay)

Files already downloaded and verified
Files already downloaded and verified
#SAMPLES: [27777], POS:NEG: [2777 : 25000], POS RATIO: 0.1000
#SAMPLES: [10000], POS:NEG: [5000 : 5000], POS RATIO: 0.5000


# **Pytorch Learning Rate Scheduling**
We will cover three scheduling functions in this section: 
*   CosineAnnealingLR
*   ReduceLROnPlateau
*   MultiStepLR

For more details, please refer to orginal PyTorch [doc](https://pytorch.org/docs/stable/optim.html).


In [7]:
def reset_model():
    # loss & optimizer
    loss_fn = AUCMLoss()
    optimizer = PESG(model, 
                    loss_fn=loss_fn,
                    lr=lr, 
                    epoch_decay=epoch_decay, 
                    margin=margin, 
                    weight_decay=weight_decay)
    return loss_fn, optimizer

### CosineAnnealingLR

In [10]:
total_epochs = 10
loss_fn, optimizer = reset_model()
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, len(trainloader)*total_epochs)

In [11]:
model.train()    
for epoch in range(total_epochs):
     for i, (data, targets) in enumerate(trainloader):
         data, targets  = data.cuda(), targets.cuda()
         y_pred = model(data)
         y_pred = torch.sigmoid(y_pred)
         loss = loss_fn(y_pred, targets)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
         scheduler.step()
     print("epoch: {}, loss: {:4f}, lr:{:4f}".format(epoch, loss.item(), optimizer.lr))          

epoch: 0, loss: 0.096573, lr:0.097575
epoch: 1, loss: 0.093474, lr:0.090493
epoch: 2, loss: 0.102245, lr:0.079448
epoch: 3, loss: 0.054700, lr:0.065520
epoch: 4, loss: 0.070871, lr:0.050072
epoch: 5, loss: 0.075728, lr:0.034618
epoch: 6, loss: 0.061491, lr:0.020669
epoch: 7, loss: 0.072430, lr:0.009592
epoch: 8, loss: 0.024156, lr:0.002470
epoch: 9, loss: 0.089273, lr:0.000000


### ReduceLROnPlateau

In [12]:
total_epochs = 20
loss_fn, optimizer = reset_model()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 
                                                       patience=3,  
                                                       verbose=True, 
                                                       factor=0.5, 
                                                       threshold=0.001,
                                                       min_lr=0.00001)

In [13]:
model.train()    
for epoch in range(total_epochs):
     for i, (data, targets) in enumerate(trainloader):
         data, targets  = data.cuda(), targets.cuda()
         y_pred = model(data)
         y_pred = torch.sigmoid(y_pred)
         loss = loss_fn(y_pred, targets)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
     scheduler.step(loss)
     print("epoch: {}, loss: {:4f}, lr:{:4f}".format(epoch, loss.item(), optimizer.lr))          

epoch: 0, loss: 0.051323, lr:0.100000
epoch: 1, loss: 0.073475, lr:0.100000
epoch: 2, loss: 0.072317, lr:0.100000
epoch: 3, loss: 0.063765, lr:0.100000
Epoch 00005: reducing learning rate of group 0 to 5.0000e-02.
epoch: 4, loss: 0.075719, lr:0.100000
epoch: 5, loss: 0.028987, lr:0.050000
epoch: 6, loss: 0.042710, lr:0.050000
epoch: 7, loss: 0.047592, lr:0.050000
epoch: 8, loss: 0.049839, lr:0.050000
epoch: 9, loss: 0.022842, lr:0.050000
epoch: 10, loss: 0.035902, lr:0.050000
epoch: 11, loss: 0.067043, lr:0.050000
epoch: 12, loss: 0.040740, lr:0.050000
Epoch 00014: reducing learning rate of group 0 to 2.5000e-02.
epoch: 13, loss: 0.039672, lr:0.050000
epoch: 14, loss: 0.023661, lr:0.025000
epoch: 15, loss: 0.031596, lr:0.025000
epoch: 16, loss: 0.054208, lr:0.025000
Epoch 00018: reducing learning rate of group 0 to 1.2500e-02.
epoch: 17, loss: 0.044520, lr:0.025000
epoch: 18, loss: 0.035632, lr:0.012500
epoch: 19, loss: 0.016720, lr:0.012500


### MultiStepLR

In [14]:
total_epochs = 20
loss_fn, optimizer = reset_model()
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10,15], gamma=0.1)

In [15]:
# reset model
model.train()    
for epoch in range(total_epochs):
     for i, (data, targets) in enumerate(trainloader):
         data, targets  = data.cuda(), targets.cuda()
         y_pred = model(data)
         y_pred = torch.sigmoid(y_pred)
         loss = loss_fn(y_pred, targets)
         optimizer.zero_grad()
         loss.backward()
         optimizer.step()
     scheduler.step()
     print("epoch: {}, loss: {:4f}, lr:{:4f}".format(epoch, loss.item(), optimizer.lr))          

epoch: 0, loss: 0.036762, lr:0.100000
epoch: 1, loss: 0.036951, lr:0.100000
epoch: 2, loss: 0.050001, lr:0.100000
epoch: 3, loss: 0.031577, lr:0.100000
epoch: 4, loss: 0.049376, lr:0.100000
epoch: 5, loss: 0.030413, lr:0.100000
epoch: 6, loss: 0.066324, lr:0.100000
epoch: 7, loss: 0.017938, lr:0.100000
epoch: 8, loss: 0.040055, lr:0.100000
epoch: 9, loss: 0.055795, lr:0.100000
epoch: 10, loss: 0.036341, lr:0.010000
epoch: 11, loss: 0.029210, lr:0.010000
epoch: 12, loss: 0.015228, lr:0.010000
epoch: 13, loss: 0.037940, lr:0.010000
epoch: 14, loss: 0.025263, lr:0.010000
epoch: 15, loss: 0.024623, lr:0.001000
epoch: 16, loss: 0.041440, lr:0.001000
epoch: 17, loss: 0.019552, lr:0.001000
epoch: 18, loss: 0.014872, lr:0.001000
epoch: 19, loss: 0.024439, lr:0.001000
